<a href="https://colab.research.google.com/github/iammawaistariq/PAI-Lab/blob/main/Lab_8_Machine_Learning_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Machine Learning Implementation

*   Traditional Implementation
*   Sklearn Pipeline Implementation

**Traditional Machine Learning Implementation**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# Load the dataset
titanic_data = pd.read_csv('titanic.csv')

#print(titanic_data.isnull().sum())
# Handle missing values
titanic_data['Age'].fillna(titanic_data['Age'].mean(), inplace=True)
titanic_data['Embarked'].fillna(titanic_data['Embarked'].mode()[0], inplace=True)

#sex = pd.get_dummies(train['Sex'],dtype=int)
titanic_data = pd.get_dummies(titanic_data, columns=['Sex', 'Embarked'], dtype=int)

# Select features and target variable
#X = titanic_data[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S']]
X = titanic_data.drop(['Survived', 'PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
y = titanic_data[['Survived']]
print(X)
print(y)

# Step 4: Split the Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Train the KNN Model
knn = KNeighborsClassifier(n_neighbors=5)  # You can tune n_neighbors
knn.fit(X_train, y_train)

# Step 7: Make Predictions
y_pred = knn.predict(X_test)
#print("Predicted Values:")
#print(y_pred )

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy:.4f}")

# Step 8: Evaluate the Model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))



     Pclass        Age  SibSp  Parch     Fare  Sex_female  Sex_male  \
0         3  22.000000      1      0   7.2500           0         1   
1         1  38.000000      1      0  71.2833           1         0   
2         3  26.000000      0      0   7.9250           1         0   
3         1  35.000000      1      0  53.1000           1         0   
4         3  35.000000      0      0   8.0500           0         1   
..      ...        ...    ...    ...      ...         ...       ...   
886       2  27.000000      0      0  13.0000           0         1   
887       1  19.000000      0      0  30.0000           1         0   
888       3  29.699118      1      2  23.4500           1         0   
889       1  26.000000      0      0  30.0000           0         1   
890       3  32.000000      0      0   7.7500           0         1   

     Embarked_C  Embarked_Q  Embarked_S  
0             0           0           1  
1             1           0           0  
2             0      

<ipython-input-9-4fa4d58b2b6f>:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  titanic_data['Age'].fillna(titanic_data['Age'].mean(), inplace=True)
<ipython-input-9-4fa4d58b2b6f>:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inpl

**Sklearn Pipeline Implementation**

In [ ]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# Load the dataset
data = pd.read_csv('titanic.csv')

#print("Original DataFrame:")
#print(data)
#print(data.shape)


def impute_embarked(X):
    X['Embarked'] = X['Embarked'].fillna(X['Embarked'].mode()[0])  # Fill missing values
    return X

# Custom function to create the FamilySize feature
def create_family_size(X):
    X['FamilySize'] = X['SibSp'] + X['Parch'] + 1  # Adding 1 for the individual themselves
    return X

# Custom function to drop specified columns
def drop_columns(X):
    return X.drop(['SibSp', 'Parch'], axis=1)

# Function to create FamilySize and drop SibSp and Parch columns
def family_size(X):
    X = create_family_size(X)
    X = drop_columns(X)
    return X

# Create pipelines for Age
age_pipeline = Pipeline(steps=[
    ('age_imputer', SimpleImputer(strategy='mean')),  # Impute Age
    ('age_scaler', MinMaxScaler())  # Scale Age
])

fare_pipeline = Pipeline(steps=[
    #('fare_imputer', SimpleImputer(strategy='mean')),  # Impute Fare
    ('fare_scaler', MinMaxScaler())  # Scale Fare
])

family_size_pipeline = Pipeline(steps=[
    ('family_size_creator', FunctionTransformer(family_size)),
    ('family_size_scaler', MinMaxScaler()),  # Scale Family_Size
])

embarked_pipeline = Pipeline(steps=[
    ('embarked_imputer', FunctionTransformer(impute_embarked)),  # Impute Embarked
    ('embarked_onehot', OneHotEncoder())  # One-hot encode Embarked
])

# Create a ColumnTransformer to preprocess the data
preprocessor = ColumnTransformer(transformers=[
    ('age_encoder', age_pipeline, ['Age']),
    ('fare_encoder', fare_pipeline, ['Fare']),
    ('family_size', family_size_pipeline, ['SibSp', 'Parch']),  # Process FamilySize
    ('embarked_encoder', embarked_pipeline, ['Embarked']),
    ('sex_encoder', OneHotEncoder(), ['Sex']),
    ('Pclass_scaler', MinMaxScaler(), ['Pclass']),  # Scale Pclass
], remainder='passthrough')

# Create a complete pipeline that includes preprocessing and the classifier
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Preprocessing steps
    ('classifier', KNeighborsClassifier(n_neighbors=5))  # KNN Classifier
])

X = data.drop(['Survived', 'PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
y = data['Survived']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train)
print(y_train)
print(type(X_train))
print(type(y_train))
print(type(X_test))
print(type(y_test))

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Make predictions on the test set
y_pred = pipeline.predict(X_test)
print("Predicted Values:")
print(y_pred)
print(y_pred.shape)
print(type(y_pred))

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy:.2f}")

# Step 8: Evaluate the Model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


     Pclass     Sex   Age  SibSp  Parch      Fare Embarked
331       1    male  45.5      0      0   28.5000        S
733       2    male  23.0      0      0   13.0000        S
382       3    male  32.0      0      0    7.9250        S
704       3    male  26.0      1      0    7.8542        S
813       3  female   6.0      4      2   31.2750        S
..      ...     ...   ...    ...    ...       ...      ...
106       3  female  21.0      0      0    7.6500        S
270       1    male   NaN      0      0   31.0000        S
860       3    male  41.0      2      0   14.1083        S
435       1  female  14.0      1      2  120.0000        S
102       1    male  21.0      0      1   77.2875        S

[712 rows x 7 columns]
331    0
733    0
382    0
704    0
813    0
      ..
106    1
270    0
860    0
435    1
102    0
Name: Survived, Length: 712, dtype: int64
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.

In [ ]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier  # Import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# Load the dataset
data = pd.read_csv('titanic.csv')

#print("Original DataFrame:")
#print(data)
#print(data.shape)

def impute_embarked(X):
    X['Embarked'] = X['Embarked'].fillna(X['Embarked'].mode()[0])  # Fill missing values
    return X

# Custom function to create the FamilySize feature
def create_family_size(X):
    X['FamilySize'] = X['SibSp'] + X['Parch'] + 1  # Adding 1 for the individual themselves
    return X

# Custom function to drop specified columns
def drop_columns(X):
    return X.drop(['SibSp', 'Parch'], axis=1)

# Function to create FamilySize and drop SibSp and Parch columns
def family_size(X):
    X = create_family_size(X)
    print(X)
    X = drop_columns(X)
    print(X)
    return X


# Create pipelines for Age
age_pipeline = Pipeline(steps=[
    ('age_imputer', SimpleImputer(strategy='mean')),  # Impute Age
    ('age_scaler', MinMaxScaler())  # Scale Age
])

fare_pipeline = Pipeline(steps=[
    #('fare_imputer', SimpleImputer(strategy='mean')),  # Impute Fare
    ('fare_scaler', MinMaxScaler())  # Scale Fare
])

family_size_pipeline = Pipeline(steps=[
    ('family_size_creator', FunctionTransformer(family_size)),
    ('family_size_scaler', MinMaxScaler()),  # Scale Family_Size
])

embarked_pipeline = Pipeline(steps=[
    ('embarked_imputer', FunctionTransformer(impute_embarked)),  # Impute Embarked
    ('embarked_onehot', OneHotEncoder())  # One-hot encode Embarked
])

# Create a ColumnTransformer to preprocess the data
preprocessor = ColumnTransformer(transformers=[
    ('age_encoder', age_pipeline, ['Age']),
    ('fare_encoder', fare_pipeline, ['Fare']),
    ('family_size', family_size_pipeline, ['SibSp', 'Parch']),  # Process FamilySize
    ('embarked_encoder', embarked_pipeline, ['Embarked']),
    ('sex_encoder', OneHotEncoder(), ['Sex']),
    ('Pclass_scaler', MinMaxScaler(), ['Pclass']),  # Scale Pclass
], remainder='passthrough')

# Create a complete pipeline that includes preprocessing and the classifier
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Preprocessing steps
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))  # RandomForest Classifier
])

X = data.drop(['Survived', 'PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
y = data['Survived']
print('-----------------')
print(X)
print(y)
print('-----------------')

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train)
print(y_train)
print(type(X_train))
print(type(y_train))
print(type(X_test))
print(type(y_test))

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Make predictions on the test set
y_pred = pipeline.predict(X_test)
print("Predicted Values:")
print(y_pred)
print(y_pred.shape)
print(type(y_pred))

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy:.2f}")

# Step 8: Evaluate the Model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


-----------------
     Pclass     Sex   Age  SibSp  Parch     Fare Embarked
0         3    male  22.0      1      0   7.2500        S
1         1  female  38.0      1      0  71.2833        C
2         3  female  26.0      0      0   7.9250        S
3         1  female  35.0      1      0  53.1000        S
4         3    male  35.0      0      0   8.0500        S
..      ...     ...   ...    ...    ...      ...      ...
886       2    male  27.0      0      0  13.0000        S
887       1  female  19.0      0      0  30.0000        S
888       3  female   NaN      1      2  23.4500        S
889       1    male  26.0      0      0  30.0000        C
890       3    male  32.0      0      0   7.7500        Q

[891 rows x 7 columns]
0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64
-----------------
     Pclass     Sex   Age  SibSp  Parch      Fare Embarked
331       1    male  45.5      0      0   28

# Lab Task

*  Extend the traditional implemenetation by including the EDA steps performed in the Sklearn Pipeline implementation.
*  Analyse the output of the EDA steps performed using the Sklearn Preprocessing and Pipeline functions and investigate whether there are any logical errors or not.
*  Apply cross-valiadation using 5-folds and display both fold wise and overall accuracy.
*  Perform the steps mentioned above on the following dataset

https://www.kaggle.com/datasets/kamilpytlak/personal-key-indicators-of-heart-disease

## Importing Libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score


## Loading dataset

In [7]:
# Load the dataset
heart_data = pd.read_csv('heart_2020_cleaned.csv')


## Checking Missing Values

In [3]:
# Step 1: Check for missing values
print("Missing values per column:")
print(heart_data.isnull().sum())

Missing values per column:
HeartDisease        0
BMI                 0
Smoking             0
AlcoholDrinking     0
Stroke              0
PhysicalHealth      0
MentalHealth        0
DiffWalking         0
Sex                 0
AgeCategory         0
Race                0
Diabetic            0
PhysicalActivity    0
GenHealth           0
SleepTime           0
Asthma              0
KidneyDisease       0
SkinCancer          0
dtype: int64


## Handilng Missing Values

In [8]:
# Step 2: Handle missing values
# Fill numeric columns with mean, categorical with mode
for col in heart_data.columns:
    if heart_data[col].dtype == 'object':
        heart_data[col] = heart_data[col].fillna(heart_data[col].mode()[0])
    else:
        heart_data[col].fillna = heart_data[col].fillna(heart_data[col].mean())

In [9]:
# Step 3: Separate label before encoding
y = heart_data['HeartDisease']
X = heart_data.drop('HeartDisease', axis=1)

In [10]:
# Step 4: Encode categorical features
X = pd.get_dummies(X, drop_first=True, dtype=int)

# Step 5: Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Train the KNN model (fix shape warning with .ravel())
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train.ravel())


<ipython-input-10-57f9be041c26>:9: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  knn.fit(X_train, y_train.ravel())


KNeighborsClassifier()

In [11]:
# Step 7: Predict
y_pred = knn.predict(X_test)


In [12]:
# Step 8: Evaluate
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy:.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))



Model Accuracy: 0.9071
Confusion Matrix:
[[57618   749]
 [ 5194   398]]
